# Discrete spatial models in BioSimulator.jl

In [ ]:
# TODO: installation script
function generate_random_2Dpoints(xlim, ylim, saturation; boundary = false)
    area = (xlim[2] - xlim[1] + 1) * (ylim[2] - ylim[1] + 1)
    n = Int(ceil(area * saturation))

    list = Tuple{Int,Int}[]
    xrange = xlim[1]:xlim[2]
    yrange = ylim[1]:ylim[2]

    while length(list) < n
        point = (rand(xrange), rand(yrange))
        point ∉ list && push!(list, point)
    end

    if boundary
        for x in xrange
            push!(list, (x, ylim[1]-1))
            push!(list, (x, ylim[2]+1))
        end

        for y in yrange
            push!(list, (xlim[1]-1, y))
            push!(list, (xlim[2]+1, y))
        end
    end

    m = boundary ? 2*(xlim[2] - xlim[1] + 1) + 2*(ylim[2] - ylim[1] + 1) : 0

    points = zeros(Int, 2, n + m)
    for i in eachindex(list)
        points[1, i] = list[i][1]
        points[2, i] = list[i][2]
    end

    return points
end

In [ ]:
using BioSimulator

## Model specification

### Base model

In [ ]:
base_model = @def_reactions begin
    Fox + 0 --> 0 + Fox, α1
    Rabbit + 0 --> 0 + Rabbit, α2
    Rabbit + 0 --> Rabbit + Rabbit, β
    Fox + Rabbit --> Fox + Fox, γ
    Rabbit --> 0, δ1
    Fox --> 0, δ2
end α1 α2 β γ δ1 δ2

### Adding lattice topology

In [ ]:
α1 = 1.0
α2 = 1.0
β = 2.0
γ = 1.5
δ1 = 1.0
δ2 = 0.5

params = [α1, α2, β, γ, δ1, δ2]

model = @enumerate_with_sclass base_model VonNeumann() 2 params

### Setting up initial conditions

In [ ]:
coord = generate_random_2Dpoints((1,100), (1,100), 0.2)
types = rand(["Fox", "Rabbit"], size(coord, 2))
state = Lattice(coord, types, nbhood = VonNeumann(), type_list = Dict(1 => "Fox", 2 => "Rabbit"))

using Plots
gr(markersize = 2)

plot(Configuration(state))

### Simulation

In [ ]:
algorithm = Direct()
tfinal = 10.0

trajectory = @time simulate(state, model, algorithm, tfinal = tfinal, save_points = 0:0.125:tfinal)

@gif for configuration in trajectory
    plot(configuration, xlim = (-100, 200), ylim = (-100, 200), markersize = 1, legend = false)
end

### Adding boundaries

In [ ]:
base_model = @def_reactions begin
    Fox + 0 --> 0 + Fox, α1
    Rabbit + 0 --> 0 + Rabbit, α2
    Rabbit + 0 --> Rabbit + Rabbit, β
    Fox + Rabbit --> Fox + Fox, γ
    Rabbit --> 0, δ1
    Fox --> 0, δ2
    X --> 0, not_used
end α1 α2 β γ δ1 δ2 not_used

In [ ]:
params = [α1, α2, β, γ, δ1, δ2, 0.0]

model = @enumerate_with_sclass base_model VonNeumann() 2 params

In [ ]:
coord = generate_random_2Dpoints((1,100), (1,100), 0.2, boundary = true)
types = rand(["Fox", "Rabbit"], size(coord, 2) - 4*100)
types = [types; ["X" for _ in 1:400]]
state = Lattice(coord, types, nbhood = VonNeumann(), type_list = Dict(1 => "Fox", 2 => "Rabbit", 3 => "X"))

using Plots
gr(markersize = 2)

plot(Configuration(state))

In [ ]:
algorithm = Direct()
tfinal = 40.0

trajectory = @time simulate(state, model, algorithm, tfinal = tfinal, save_points = 0:0.125:tfinal)

@gif for configuration in trajectory
    plot(configuration, xlim = (-1, 102), ylim = (-1, 102), markersize = 1.25, legend = false)
end every 2

In [ ]:
population = zeros(Int, length(trajectory), 2)

for (i, config) in enumerate(trajectory)
    # count foxes
    population[i, 1] = count(isequal(2), config.tcode)

    # count rabbits
    population[i, 2] = count(isequal(3), config.tcode)
end

plot(trajectory.t, population, xlabel = "time (A.U.)", ylabel = "count", label = ["fox" "rabbit"])